# Merge da juncao de logs com o arquvio randomizado de frases

In [1]:
import pandas as pd # type: ignore
import csv

# Caminhos dos arquivos de entrada
log_path = "dados/MQD_1465_log_semfrases.csv"
randomizado_path = "dados/MQD_1465_randomizado.csv"

# Leitura segura do arquivo randomizado com detecção de delimitador
with open(randomizado_path, "r", encoding="utf-8") as f:
    sample = f.read(2048)

sniffer = csv.Sniffer()
dialect = sniffer.sniff(sample)

df_randomizado = pd.read_csv(randomizado_path, sep=dialect.delimiter)
df_log = pd.read_csv(log_path)

# Garante a coluna ID_Random
if "ID_Random" not in df_randomizado.columns:
    df_randomizado.insert(0, "ID_Random", range(1, len(df_randomizado) + 1))

# Merge pelo índice
df_log_completo = pd.merge(
    df_log,
    df_randomizado[["ID_Random", "frase"]],
    left_on="ItemNumber",
    right_on="ID_Random",
    how="left"
)

# Reorganiza as colunas
colunas = df_log_completo.columns.tolist()
idx_item = colunas.index("ItemNumber")
colunas_organizadas = colunas[:idx_item + 1] + ["frase"] + [col for col in colunas if col not in ["ItemNumber", "frase", "ID_Random"]]
df_log_completo = df_log_completo[colunas_organizadas]

# Salva o CSV com aspas apenas na coluna 'frase'
output_path = "dados/MQD_1465_final.csv"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(",".join(df_log_completo.columns.tolist()) + "\n")
    for _, row in df_log_completo.iterrows():
        linha = []
        for col in df_log_completo.columns:
            val = row[col]
            if col == "frase":
                val = str(val)
                if not (val.startswith('"') and val.endswith('"')):
                    val = f'"{val}"'
            linha.append(str(val))
        f.write(",".join(linha) + "\n")

print("Arquivo salvo em:", output_path)

Arquivo salvo em: dados/MQD_1465_final.csv
